In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
train_data_dir = 'Chili_Plant_Disease/train'
valid_data_dir = 'Chili_Plant_Disease/val'

In [3]:
img_width, img_height = 224, 224
batch_size = 32

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [5]:
valid_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 161 images belonging to 2 classes.


In [7]:
valid_generator = valid_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 20 images belonging to 2 classes.


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])


In [10]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [11]:
epochs = 10
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    epochs=epochs
)

Epoch 1/10
5/5 [==============================] - 16s 2s/step - loss: 1.8153 - accuracy: 0.4341
Epoch 2/10
5/5 [==============================] - 9s 2s/step - loss: 0.6922 - accuracy: 0.5581
Epoch 3/10
5/5 [==============================] - 10s 2s/step - loss: 0.6442 - accuracy: 0.5659
Epoch 4/10
5/5 [==============================] - 11s 2s/step - loss: 0.5625 - accuracy: 0.7250
Epoch 5/10
5/5 [==============================] - 10s 2s/step - loss: 0.5144 - accuracy: 0.8062
Epoch 6/10
5/5 [==============================] - 10s 2s/step - loss: 0.5661 - accuracy: 0.7519
Epoch 7/10
5/5 [==============================] - 11s 2s/step - loss: 0.5899 - accuracy: 0.7000
Epoch 8/10
5/5 [==============================] - 10s 2s/step - loss: 0.5085 - accuracy: 0.7984
Epoch 9/10
5/5 [==============================] - 10s 2s/step - loss: 0.4560 - accuracy: 0.8527
Epoch 10/10
5/5 [==============================] - 9s 2s/step - loss: 0.4196 - accuracy: 0.8372


In [12]:
model.save('plant_health_model.h5')

testing


In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

In [14]:
model = tf.keras.models.load_model('plant_health_model.h5')

In [54]:
test_image_path = 'Demo3.jpg'

In [55]:
img_width, img_height = 224, 224

In [56]:
test_image = load_img(test_image_path, target_size=(img_width, img_height))
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image = test_image / 255.0

In [57]:
predictions = model.predict(test_image)

1/1 [==============================] - 0s 90ms/step


In [58]:
predicted_class = 'Healthy' if predictions[0][0] < 0.5 else 'Unhealthy'

In [59]:
prediction_probability = predictions[0][0] if predicted_class == 'Unhealthy' else 1 - predictions[0][0]

In [60]:
print('Predicted Class:', predicted_class)
print('Prediction Probability:', prediction_probability)

Predicted Class: Unhealthy
Prediction Probability: 0.6356472
